# Prepare a vacuum scan for subtraction from other scans
 - Once completed, use vacuum_scan_subtract.ipynb to apply the vacuum scan subtraction to other scans
 - Each scan must have the same STEM magnification and camera length.

In [ ]:
%matplotlib widget

from pathlib import Path
from collections import namedtuple

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm,PowerNorm
from matplotlib.patches import Circle
import numpy as np
from numpy.linalg import svd
import imageio
from scipy import ndimage

import stempy.io as stio
import stempy.image as stim

import ncempy

def planeFit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.

    """

    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

def get_scan_path(directory, scan_num=None, scan_id=None, th=None):
    """ Get the file path for a 4D Camera scan using the scan number or 
    the Distiller scan id. The threshold is optional. scan_id should always
    be unique and is the best option to load a dataset.
    
    A ValueError is raised if more than one file matches the input. Then the user
    needs to input more information to narrow down the choices.
    
    Parameters
    ----------
    directory : pathlib.Path or str
        The path to the directory containing the file.
    scan_id : int, optional
        The Distiller scan id.
    scan_num : int
        The 4D Camera scan number. Optional
    th : float, optional
        The threshold for counting. This is added to the filename in some cases.
        
    Returns
    -------
    : tuple
        The tuple contains the file that matches the input information and the 
        scan_num and scan_id as a tuple.
    """
    if scan_id is not None:
        # This should be unique
        file_path = list(directory.glob(f'*_id{scan_id}*.h5'))
    elif scan_num is not None:
        # older files might include the threshold (th)
        if th is not None:
            file_path = list(directory.glob('data_scan{}_th{}_electrons.h5'.format(scan_num, th)))
        else:
            file_path = list(directory.glob('data_scan{}*electrons.h5'.format(scan_num)))
    else:
        raise TypeError('Missing scan_num or scan_id input.')

    if len(file_path) > 1:
        raise ValueError('Multiple files match that input. Add scan_id to be more specific.')
    elif len(file_path) == 1:
        file_path = file_path[0]
        # Determine the scan_id and scan_num for use later (i.e. getting DM4 file)
        spl = file_path.name.split('_')
        for ii in spl:
            if 'id' in ii:
                scan_id = int(ii[len('id'):])
            elif 'scan' in ii:
                scan_num = int(ii[len('scan'):])
    else:
        raise FileNotFoundError('No file with those parameters can be found.')
    return file_path, scan_num, scan_id

distiller_path = Path('/global/cfs/cdirs/ncemhub/distiller/')

# Load the vacuum scan data set

In [ ]:
# Close all previous windows to avoid too many windows
plt.close('all')

date = Path('2023.01.13')
scan_id = {{ scan.id }}
scan_num = {{ scan.scan_id }}
th = None # optional; use None or integer

fname, scan_num, scan_id = get_scan_path(distiller_path / Path('counted') / Path(date), scan_num, scan_id=scan_id)

# Load the sparse data
vacuum_scan = stio.load_electron_counts(fname)
# Remove the flyback
vacuum_scan = vacuum_scan[:, :-1, :, :]

print('File: {}'.format(fname))
print('Scan dimensions = {}'.format(vacuum_scan.scan_shape))

# Load the HAADF (if available)
haadf_path = distiller_path / Path('dm4') / Path(date) / Path(f'scan{scan_num}.dm4')

if haadf_path.exists():
    with ncempy.io.dm.fileDM(haadf_path) as f0:
        md = f0.getMetadata(0)
        camera_length = md['Microscope Info STEM Camera Length']
        stem_magnification = md['Microscope Info Indicated Magnification']
        print(f'Camera length = {camera_length} mm')
        print(f'STEM magnification = {stem_magnification} kx')
else:
    print('No HAADF available')


# Find the center of the center beam at each scan position
 - $center$ variable should be the average x and y position of the beam
 - $radius$ variable should contain all possible positions of the beam. All electrons outside this radius will be ignored

In [ ]:
# Calculate the summed diffraction pattern to set hyperparameters
dp = vacuum_scan.sum(axis=(0, 1))

# Get a single dense diffraction pattern from the center
dp0 = vacuum_scan[vacuum_scan.scan_shape[0]//2, vacuum_scan.scan_shape[1]//2, :, :]

# Set the center of the pattern (use figure below for manual)
center = stim.com_dense(dp) # use the center of mass of the summed pattern
center = (290, 293) # manually define as column, row from plot (if automated technique fails)
print(f'Average center of beam = {center}')

# Define the radius for iterative processing
radius = 40 # pixels

fg,ax = plt.subplots(1, 2, sharex=True, sharey=True)
ax[1].imshow(dp, norm=PowerNorm(0.75))
ax[1].scatter(center[0], center[1], c='r')
ax[1].add_patch(Circle(center, radius=radius, fc=None, fill=None, ec='r'))
ax[0].imshow(dp0, norm=PowerNorm(0.75))
ax[0].set(title='single pattern')
ax[1].set(title='summed pattern')
ax[1].legend(['center of pattern', 'crop_to radius']);

In [ ]:
# Calculate the center of mass (COM) iteratively for each scan position
com2 = stim.com_sparse(vacuum_scan, crop_to=(radius, radius), init_center=center[::-1])

fg,ax = plt.subplots(1, 2)
ax[0].imshow(com2[0,]-com2_median[0], vmin=-com2_std[0]*3, vmax=com2_std[0]*3, cmap='bwr')
ax[0].set(title='COM: axis 0')
ax[1].imshow(com2[1,]-com2_median[1], vmin=-com2_std[1]*3, vmax=com2_std[1]*3, cmap='bwr')
ax[1].set(title='COM: axis 1')

fg,ax = plt.subplots(1, 2)
ax[0].hist(com2[0,].ravel() - com2_median[0])
ax[0].set(title='COM: axis 0')
ax[1].hist(com2[1,].ravel()-com2_median[1])
ax[1].set(title='COM: axis 1');

In [ ]:
# Remove the outliers by median filtering
com2_filt = np.zeros_like(com2)
com2_filt[0,] = ndimage.median_filter(com2[0,], size=(5, 5))
com2_filt[1,] = ndimage.median_filter(com2[1,], size=(5, 5))

com2_median = np.median(com2_filt, axis=(1, 2))

fg,ax = plt.subplots(1, 2,sharex=True,sharey=True)
ax[0].imshow(com2_filt[0,]-com2_median[0],cmap='bwr',vmin=-3,vmax=3)
ax[0].set(title='COM: axis 0')
ax[1].imshow(com2_filt[1,]-com2_median[1],cmap='bwr',vmin=-3,vmax=3);
ax[1].set(title='COM: axis 1');

## Fit the beam motion to reduce noise
 - If the motion is linear then a plane is sufficient
 - If the motion is non-linear then use interpolation

In [ ]:
# Interpolate to fit the vacuum scan_shape

YY, XX = np.mgrid[0:com2.shape[1], 0:com2.shape[2]]

# Interpolation
com2_fit = np.zeros((2, *XX.shape))
com2_fit[0,:,:] = ndimage.map_coordinates(com2_filt[0,:,:], (YY.ravel(), XX.ravel()),mode='nearest').reshape(XX.shape)
com2_fit[1,:,:] = ndimage.map_coordinates(com2_filt[1,:,:], (YY.ravel(), XX.ravel()),mode='nearest').reshape(XX.shape)

com2_fit_median = np.median(com2_fit,axis=(1,2))

# Fit to a plane
planeCOM0 = planeFit(np.stack((YY, XX, com2_filt[0,]-com2_median[0])))
planeCOM1 = planeFit(np.stack((YY, XX, com2_filt[1,]-com2_median[1])))

print(f'plane fit to COM0: {planeCOM0}')
print(f'plane fit to COM1: {planeCOM1}')

normal = planeCOM0[1]
d = np.dot(-planeCOM0[0], normal)
# calculate corresponding z
z0 = (-normal[0]*YY - normal[1]*XX - d)/normal[2]

normal = planeCOM1[1]
d = np.dot(-planeCOM1[0], normal)
# calculate corresponding z
z1 = (-normal[0]*YY - normal[1]*XX - d)/normal[2]

# Plot everything together
fg,ax = plt.subplots(2, 3)
ax[0, 0].imshow(com2_filt[0,],cmap='bwr')
ax[0, 0].set(title='Experimental')
ax[0, 1].imshow(z0, cmap='bwr')
ax[0, 1].set(title='Linear plane fit')
ax[1, 0].imshow(com2_filt[1,],cmap='bwr')
ax[1, 1].imshow(z1, cmap='bwr')
ax[0, 2].imshow(com2_fit[0,]-com2_fit_median[0], cmap='bwr')
ax[1, 2].imshow(com2_fit[1,]-com2_fit_median[1], cmap='bwr')
ax[0, 2].set(title='Interpolation')
fg.tight_layout()

## Remove shifts from the vacuum scan to test the fit
 - Set the $method$ variable according to the type of shift you want to use in the correction (usually interp)

In [ ]:
# Test centering on the vacuum scan itself
method = 'interp' # choose plane or interp

if method == 'interp':
    scan_origin0 = com2_fit[0,]
    scan_origin1 = com2_fit[1,]
elif method == 'plane':
    scan_origin0 = z0
    scan_origin1 = z1
else:
    raise ValueError(f"Incorrrect method: {method}")
scan_origin0_round = np.round(scan_origin0).astype(np.int32) - int(scan_origin0.mean())
scan_origin1_round = np.round(scan_origin1).astype(np.int32) - int(scan_origin1.mean())

# Ensure the data sets are the same size
assert vacuum_scan.data.shape[0] == scan_origin0_round.ravel().shape[0]

centered = np.empty_like(vacuum_scan.data)
for ii, (eev, x, y) in enumerate(zip(vacuum_scan.data, scan_origin0_round.ravel(), scan_origin1_round.ravel())):
    for jj, ev in enumerate(eev):
        evx, evy = np.unravel_index(ev, vacuum_scan.frame_shape)
        evx_centered = evx - y # need to flip x and y
        evy_centered = evy - x

        # Some events will get pushed off the detector by the shift. Remove them
        keep = (evx_centered < vacuum_scan.frame_shape[0]) & (evx_centered >= 0) & (evy_centered < vacuum_scan.frame_shape[1]) & (evy_centered >= 0)
        evx_centered = evx_centered[keep]
        evy_centered = evy_centered[keep]

        centered[ii, jj] = np.ravel_multi_index((evx_centered,evy_centered), vacuum_scan.frame_shape)

vacuum_scan_centered = stio.SparseArray(centered, vacuum_scan.scan_shape, vacuum_scan.frame_shape)

dp = vacuum_scan.sum(axis=(0,1))
dp2 = vacuum_scan_centered.sum(axis=(0,1))

fg,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(dp)
ax[0].set(title='Summed raw data')
ax[1].imshow(dp2)
ax[1].set(title='Summed centered data');

In [ ]:
# Save the data for use on other datasets

# Save the centered vacuum scan to a stempy dataset
scan_out_path = fname.with_name(fname.stem + '_centered.h5')
vacuum_scan_centered.write_to_hdf5(scan_out_path)
print(f'Centered scan saved to: {scan_out_path}')

# Save the scan shift data as an EMD file
offsets_out_path = fname.with_name(fname.stem + '_offsets.emd')

if offsets_out_path.exists():
    raise FileExistsError(f'Offsets file {offsets_out_path} already exists.\nDelete and resave to update the file')
else:
    with ncempy.io.emd.fileEMD(offsets_out_path, readonly=False) as f0:
        dims0 = ncempy.io.emd.defaultDims(com2_filt)
        f0.put_emdgroup('vacuum_COM', com2_filt, dims0)
    print(f'Offsets saved to: {offsets_out_path}')
